In [1]:
import numpy as np
import randoy =
import syft as sy
import json
from syft.core.frameworks.torch import utils

In [2]:
x = sy.array([1,2,3,4])
me = sy.local_worker
hook = me.hook

In [3]:
bob = sy.VirtualWorker(hook=hook,id=2)

In [4]:
me.add_worker(bob)

In [5]:
x = sy.array([1,2,3,4])

In [6]:
xt = x.send(bob)
# xt = x.torch().send(bob)

In [7]:
cmd, locations, owners = utils.compile_command(attr="torch", args={}, kwargs={}, has_self=True, self=xt)
cmd

{'mode': 'subscribe',
 'obj': {'args': {},
  'command': 'torch',
  'has_self': True,
  'kwargs': {},
  'self': {'data': ['data is remote'],
   'id': 10330136378,
   'id_at_location': 40560575,
   'location': 2,
   'type': 'numpy.array_ptr'}}}

In [8]:
x = me.send_command(recipient=xt.location, message=cmd, framework="numpy")

In [10]:
y = x + x

In [12]:
y.get()


 2
 4
 6
 8
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [13]:
bob._objects

{4471197629: array([1, 2, 3, 4]),
 7798415103: [_LocalTensor - id:7798415103 owner:2],
 8250913666: [_LocalTensor - id:8250913666 owner:2],
 8617114141: array([1, 2, 3, 4])}

In [15]:
x.get()

enforcing owner:<syft.core.workers.virtual.VirtualWorker id:2>
<class 'syft.core.frameworks.torch.tensor._LocalTensor'>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:2>
<class 'syft.core.frameworks.torch.tensor.FloatTensor'>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:0>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:0>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:0>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:0>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:0>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:0>
<class 'syft.core.frameworks.torch.tensor.FloatTensor'>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:0>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:0>
<class 'syft.core.frameworks.torch.tensor.DoubleTensor'>
enforcing owner:<syft.core.workers.virtual.VirtualWorker id:0>
<class 'syft.core.frameworks.torch.tensor._LocalTensor'>
enforcing owner:<syft.co


 1
 2
 3
 4
[syft.core.frameworks.torch.tensor.FloatTensor of size 4]

In [12]:
bob._objects

{5220922905: array([1, 2, 3, 4])}

In [8]:
xptr = x.send(bob)

message wrapper json
b'{"message": {"obj": {"type": "numpy.array", "id": 9550542501, "data": [1, 2, 3, 4]}, "mode": "acquire"}, "type": "obj"}\n'
{'message': array([1, 2, 3, 4]), 'type': 'obj'}


In [23]:
xptr

array_ptr(['data is remote'], dtype='<U14')

In [20]:
x = xptr.get()

In [3]:
x

array([1, 2, 3, 4])